# IBM microbiome simulation demo (standalone, non-reservoir)

This notebook demonstrates the individual-based microbiome (IBM) model used in `computingMicrobiome`, **only as a dynamical system**, not yet as a reservoir.

We will:
- Build and inspect small IBM grids (`GridState`).
- Run simulations over time using the low-level `tick` function.
- Visualize species and resource dynamics (time series and simple animations).
- Use interactive widgets to explore how changing parameters (number of species, resources, diffusion, dilution, etc.) affects the dynamics.

The notebook is designed to run in **Google Colab** from a fresh runtime.

## Environment setup (Colab)

If you are running this in **Google Colab** from a fresh runtime, run the following cell first to clone the repository and install the package.

> Note: If you are running locally inside the `computingMicrobiome` repo with the package already installed, you can skip this cell.

In [ ]:
# Uncomment this cell if running in Google Colab from a fresh runtime.
# It will clone the repo and install the package (including extras).

# !git clone https://github.com/danielriosgarza/computingMicrobiome.git
# %cd computingMicrobiome
# !uv pip install .[all]

## Imports and basic setup

We import the IBM core components (`GridState`, `EnvParams`, `SpeciesParams`, `load_params`, `make_zero_state`, `tick`) and the plotting / widget libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from numpy.random import default_rng

import ipywidgets as widgets

from computingMicrobiome.ibm import EnvParams, SpeciesParams, GridState, load_params
from computingMicrobiome.ibm.state import make_zero_state
from computingMicrobiome.ibm.stepper import tick

plt.style.use("seaborn-v0_8-darkgrid")
rng = default_rng(0)

## IBM state initialization helpers

We define helper functions to:
- Build default/environment parameters for a small grid.
- Initialize a `GridState` in different ways (empty, basal pattern, random).

These mirror the logic used in the IBM reservoir backend but keep everything focused on the raw IBM dynamics.

In [ ]:
def make_default_config(
    *,
    height: int = 16,
    width_grid: int = 32,
    n_species: int = 3,
    n_resources: int = 2,
    basal_init: bool = True,
    basal_occupancy: float = 0.6,
    basal_energy: int = 12,
    basal_resource: int = 10,
    diff_numer: int = 1,
    diff_denom: int = 8,
    dilution_p: float = 0.01,
    feed_rate: float = 1.0,
) -> dict:
    """Construct a simple IBM config dictionary.

    This is passed to `load_params` to obtain `EnvParams` and `SpeciesParams`.
    """
    cfg = dict(
        height=height,
        width_grid=width_grid,
        n_species=n_species,
        n_resources=n_resources,
        basal_init=basal_init,
        basal_occupancy=basal_occupancy,
        basal_energy=basal_energy,
        basal_resource=basal_resource,
        diff_numer=diff_numer,
        diff_denom=diff_denom,
        dilution_p=dilution_p,
        feed_rate=feed_rate,
    )
    return cfg


def build_env_species(config_overrides: dict | None = None) -> tuple[EnvParams, SpeciesParams]:
    """Create `EnvParams` and `SpeciesParams` from defaults + overrides."""
    cfg = make_default_config()
    if config_overrides is not None:
        cfg.update(config_overrides)
    env, species = load_params(cfg)
    return env, species


def init_state(env: EnvParams, mode: str = "basal", rng: np.random.Generator | None = None) -> GridState:
    """Initialize a `GridState` for the given environment.

    Modes:
    - "empty": all cells empty, resources zero.
    - "basal": deterministic pattern based on `basal_*` settings.
    - "random": random occupancy, energies, and resources.
    """
    if rng is None:
        rng = default_rng()

    if mode == "empty":
        return make_zero_state(
            height=env.height,
            width_grid=env.width_grid,
            n_resources=env.n_resources,
        )

    if mode == "basal":
        rr, cc = np.indices((env.height, env.width_grid))
        if env.basal_pattern == "stripes":
            sid = (rr % env.n_species).astype(np.int16)
        else:
            sid = ((rr + cc) % env.n_species).astype(np.int16)

        if env.basal_occupancy >= 1.0:
            occupied = np.ones((env.height, env.width_grid), dtype=bool)
        elif env.basal_occupancy <= 0.0:
            occupied = np.zeros((env.height, env.width_grid), dtype=bool)
        else:
            # Deterministic occupancy mask for reproducible runs.
            key = (rr * 73856093 + cc * 19349663) % 1000
            occupied = key < int(env.basal_occupancy * 1000.0)

        occ = np.full((env.height, env.width_grid), -1, dtype=np.int16)
        occ[occupied] = sid[occupied]

        E = np.zeros((env.height, env.width_grid), dtype=np.uint8)
        if env.basal_energy > 0:
            E[occupied] = np.uint8(env.basal_energy)

        R = np.full(
            (env.n_resources, env.height, env.width_grid),
            np.uint8(env.basal_resource),
            dtype=np.uint8,
        )
        return GridState(occ=occ, E=E, R=R)

    if mode == "random":
        occ = np.full((env.height, env.width_grid), -1, dtype=np.int16)
        occupied = rng.random((env.height, env.width_grid)) < 0.5
        sid = rng.integers(
            0,
            env.n_species,
            size=(env.height, env.width_grid),
            dtype=np.int16,
        )
        occ[occupied] = sid[occupied]

        E = np.zeros((env.height, env.width_grid), dtype=np.uint8)
        if np.any(occupied):
            rand_e = rng.integers(
                1,
                env.Emax + 1,
                size=int(np.count_nonzero(occupied)),
                dtype=np.uint16,
            ).astype(np.uint8)
            E[occupied] = rand_e

        R = rng.integers(
            0,
            env.Rmax + 1,
            size=(env.n_resources, env.height, env.width_grid),
            dtype=np.uint16,
        ).astype(np.uint8)
        return GridState(occ=occ, E=E, R=R)

    raise ValueError("mode must be one of {'empty', 'basal', 'random'}")

## Inspect a small IBM state

We now create a small grid, initialize it, and look at the arrays that represent:
- `occ`: occupancy (species id or -1 for empty)
- `E`: per-cell energy
- `R`: per-resource field over the grid

In [ ]:
env, species = build_env_species({"height": 8, "width_grid": 16, "n_species": 3, "n_resources": 2})
state = init_state(env, mode="basal", rng=rng)

print("Grid shape (H, W):", state.occ.shape)
print("Energy shape (H, W):", state.E.shape)
print("Resources shape (M, H, W):", state.R.shape)

plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.title("Species occupancy (occ)")
im0 = plt.imshow(state.occ, cmap="tab20", vmin=-1, vmax=env.n_species - 1)
plt.colorbar(im0, label="species id")

plt.subplot(1, 2, 2)
plt.title("Energy (E)")
im1 = plt.imshow(state.E, cmap="viridis")
plt.colorbar(im1, label="energy")
plt.tight_layout()
plt.show()

## Running the IBM over time

We define a helper to run the IBM for a number of ticks and record summary observables:
- Species counts over time.
- Total resources per resource type over time.

In [ ]:
def run_simulation(
    env: EnvParams,
    species: SpeciesParams,
    state: GridState,
    *,
    n_steps: int = 200,
    rng: np.random.Generator | None = None,
):
    """Run the IBM for `n_steps`, returning species and resource trajectories.

    Returns
    -------
    species_counts : array, shape (n_steps+1, n_species)
        Number of occupied cells per species at each time.
    resource_totals : array, shape (n_steps+1, n_resources)
        Total resource amount per resource type at each time.
    """
    if rng is None:
        rng = default_rng()

    species_counts = np.zeros((n_steps + 1, env.n_species), dtype=np.int32)
    resource_totals = np.zeros((n_steps + 1, env.n_resources), dtype=np.int32)

    def measure(t_idx: int) -> None:
        occ = state.occ
        for s in range(env.n_species):
            species_counts[t_idx, s] = np.count_nonzero(occ == s)
        R = state.R.reshape(env.n_resources, -1)
        resource_totals[t_idx] = R.sum(axis=1)

    measure(0)
    for t in range(1, n_steps + 1):
        tick(state, env, species, rng)
        measure(t)

    return species_counts, resource_totals


def plot_time_series(species_counts: np.ndarray, resource_totals: np.ndarray) -> None:
    """Plot species and resource trajectories over time."""
    T = species_counts.shape[0] - 1
    t = np.arange(T + 1)

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    ax = axes[0]
    for s in range(species_counts.shape[1]):
        ax.plot(t, species_counts[:, s], label=f"species {s}")
    ax.set_xlabel("time step")
    ax.set_ylabel("# occupied cells")
    ax.set_title("Species counts over time")
    ax.legend(loc="best")

    ax = axes[1]
    for m in range(resource_totals.shape[1]):
        ax.plot(t, resource_totals[:, m], label=f"resource {m}")
    ax.set_xlabel("time step")
    ax.set_ylabel("total resource")
    ax.set_title("Resource totals over time")
    ax.legend(loc="best")

    plt.tight_layout()
    plt.show()

### Example: basic IBM run

We now run the IBM for a few hundred steps and plot the resulting trajectories.

In [ ]:
env, species = build_env_species({"height": 8, "width_grid": 32, "n_species": 3, "n_resources": 2})
state = init_state(env, mode="basal", rng=default_rng(1))
species_counts, resource_totals = run_simulation(env, species, state, n_steps=200, rng=default_rng(2))
plot_time_series(species_counts, resource_totals)

## Animation of spatial dynamics

Next we build a simple animation that shows how occupancy and energy fields evolve over time on the lattice.

In [ ]:
def make_spatial_animation(
    env: EnvParams,
    species: SpeciesParams,
    *,
    n_steps: int = 100,
    init_mode: str = "basal",
    seed: int = 0,
    interval_ms: int = 100,
):
    """Create a matplotlib animation of IBM spatial dynamics.

    Returns an `HTML` object embedding the animation (suitable for notebooks).
    """
    rng_local = default_rng(seed)
    state = init_state(env, mode=init_mode, rng=rng_local)

    # Pre-allocate storage for frames.
    occ_frames = np.zeros((n_steps + 1, env.height, env.width_grid), dtype=np.int16)
    E_frames = np.zeros((n_steps + 1, env.height, env.width_grid), dtype=np.uint8)

    def snapshot(t_idx: int) -> None:
        occ_frames[t_idx] = state.occ
        E_frames[t_idx] = state.E

    snapshot(0)
    for t in range(1, n_steps + 1):
        tick(state, env, species, rng_local)
        snapshot(t)

    fig, axes = plt.subplots(1, 2, figsize=(8, 3))
    ax_occ, ax_E = axes

    im_occ = ax_occ.imshow(occ_frames[0], cmap="tab20", vmin=-1, vmax=env.n_species - 1)
    ax_occ.set_title("occupancy (species id)")

    im_E = ax_E.imshow(E_frames[0], cmap="viridis", vmin=0, vmax=env.Emax)
    ax_E.set_title("energy (E)")

    plt.tight_layout()

    def init():
        im_occ.set_data(occ_frames[0])
        im_E.set_data(E_frames[0])
        return [im_occ, im_E]

    def update(frame_idx: int):
        im_occ.set_data(occ_frames[frame_idx])
        im_E.set_data(E_frames[frame_idx])
        return [im_occ, im_E]

    anim = animation.FuncAnimation(
        fig,
        update,
        init_func=init,
        frames=n_steps + 1,
        interval=interval_ms,
        blit=True,
    )

    plt.close(fig)
    return HTML(anim.to_jshtml())

In [ ]:
# Example animation for a small grid (may take a couple of seconds to render).
env_anim, species_anim = build_env_species({"height": 8, "width_grid": 32, "n_species": 3, "n_resources": 2})
make_spatial_animation(env_anim, species_anim, n_steps=60, init_mode="basal", seed=3, interval_ms=120)

## Interactive exploration with widgets

Finally, we expose a few key IBM parameters through widgets so you can quickly explore how they affect the dynamics:
- Number of species and resources.
- Diffusion and dilution parameters.
- Grid size and number of steps.

For responsiveness, we keep default runs relatively small.

In [ ]:
def simulate_and_plot(
    height: int = 8,
    width_grid: int = 24,
    n_species: int = 3,
    n_resources: int = 2,
    diff_numer: int = 1,
    diff_denom: int = 8,
    dilution_p: float = 0.01,
    feed_rate: float = 1.0,
    n_steps: int = 150,
    init_mode: str = "basal",
    seed: int = 0,
):
    cfg = make_default_config(
        height=height,
        width_grid=width_grid,
        n_species=n_species,
        n_resources=n_resources,
        diff_numer=diff_numer,
        diff_denom=diff_denom,
        dilution_p=dilution_p,
        feed_rate=feed_rate,
    )
    env, species = load_params(cfg)
    rng_local = default_rng(seed)
    state = init_state(env, mode=init_mode, rng=rng_local)
    species_counts, resource_totals = run_simulation(env, species, state, n_steps=n_steps, rng=rng_local)
    plot_time_series(species_counts, resource_totals)


widgets.interact(
    simulate_and_plot,
    height=widgets.IntSlider(min=4, max=32, step=2, value=8, description="height"),
    width_grid=widgets.IntSlider(min=8, max=64, step=4, value=24, description="width"),
    n_species=widgets.IntSlider(min=1, max=6, step=1, value=3, description="#species"),
    n_resources=widgets.IntSlider(min=1, max=4, step=1, value=2, description="#resources"),
    diff_numer=widgets.IntSlider(min=0, max=4, step=1, value=1, description="diff_numer"),
    diff_denom=widgets.IntSlider(min=1, max=16, step=1, value=8, description="diff_denom"),
    dilution_p=widgets.FloatSlider(min=0.0, max=0.2, step=0.005, value=0.01, description="dilution_p"),
    feed_rate=widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=1.0, description="feed_rate"),
    n_steps=widgets.IntSlider(min=20, max=400, step=10, value=150, description="steps"),
    init_mode=widgets.Dropdown(options=["basal", "random", "empty"], value="basal", description="init"),
    seed=widgets.IntSlider(min=0, max=1000, step=1, value=0, description="seed"),
);

## IBM internals recap

This notebook has treated the IBM as a dynamical system in its own right. The key components are:
- `GridState(occ, E, R)`: the discrete lattice state (occupancy, energy, resources).
- `EnvParams` / `SpeciesParams`: lattice and per-species parameters parsed from a config.
- `tick(state, env, species, rng)`: one synchronous IBM update (diffusion, dilution, metabolism, reproduction).

In the full `computingMicrobiome` project, the IBM is wrapped by a reservoir backend that encodes `GridState` into feature vectors for readouts. Here we focused purely on its **ecological dynamics**, which can be used as a starting point for new experiments or teaching demos.